# Tenencia de acciones

## Importo las librerias necesarias

In [14]:
from google.cloud import firestore
import os
import pandas as pd

Guardo como variable el acceso a la base de datos de Firestore y lo inicializo

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='../acciones-usd-real-firebase-adminsdk-rfxvv-c888549652.json'

# Inicializo firestore
db = firestore.Client()

Hago la consulta a la base y traigo todos los tickers, junto con su PPC y la cantidad de cada uno, luego los convierto en un DataFrame

In [16]:
list_tickers = []
try:
    # Consulto los tickers y los guardo en una lista
    tickers = db.collection('acciones').stream()
    for ticker in tickers:
        list_tickers.append(ticker.to_dict())
    
    # Convierto la lista en un data frame
    df_tickers = pd.DataFrame(list_tickers)
    
except Exception as e:
    print(f'No fue posible consultar los datos, error: {e}')


df_tickers.head()

,Ticker,PPC,es_pesos,Cantidad,ratio
0,AGRO.BA,59.70,True,280,NaN
1,AMZN,200.70,False,35,144.0
2,COME.BA,96.80,True,741,NaN
3,GGAL.BA,3035.00,True,7,NaN
4,GOOGL,165.88,False,17,58.0


Consulto el valor del dola CCL al día de hoy utilizando la api de mercados de ambito

In [17]:
from datetime import datetime
import requests

# Funcion para obtener el valor del dolcar CCL al dia de la fecha
def valor_ccl():
    FECHA_INICIO = datetime.now()
    FECHA_FINAL = datetime.now()
    # URL de la API
    url_ccl = f"https://mercados.ambito.com//dolarrava/cl/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}"
    #url_mep = f'https://mercados.ambito.com//dolarrava/mep/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}'

    # Realizar una solicitud GET
    response = requests.get(url_ccl)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        # Parsear el contenido JSON de la respuesta
        data = response.json()
        return data[1][1]
    else:
        print(f"Error: {response.status_code}")

dolar_ccl = valor_ccl()
#dolar_ccl = 1480
print(dolar_ccl)

IndexError: list index out of range

Se realiza la descarga de la información de los tickers, quedandonos unicamente con el precio de cierre.
Se realizan distintas operaciones agregando columnas para saber lo total invertido de cada acción tanto en USD como en ARS.

In [27]:
import yfinance as yf

# Las convierto en lista para poder descargar sus datos y los descargo con yfinance
list_tickers = df_tickers['Ticker'].to_list()
yf_t = yf.download(list_tickers, period='1d', interval='2m')['Adj Close']

# Elijo el ultimo registro
yf_t = yf_t.iloc[-1]

# Creo una columna donde agrego esos precios mapeandolos con el indice de la serie anterior ya que son el mismo
df_tickers['Precio_Cierre'] = df_tickers['Ticker'].map(yf_t).round(2)

# Separo las acciones de los CEDEARS
df_acciones = df_tickers[df_tickers['es_pesos'] == True]
df_cedears = df_tickers[df_tickers['es_pesos'] == False]
df_cedears

[*********************100%%**********************]  11 of 11 completed


,Ticker,PPC,es_pesos,Cantidad,ratio,Precio_Cierre
1,AMZN,200.70,False,35,144.0,167.83
4,GOOGL,165.88,False,17,58.0,166.68
5,MELI,1641.33,False,8,120.0,1777.71
6,META,368.76,False,9,24.0,488.20
7,NVDA,121.00,False,21,24.0,107.31
9,PFE,26.20,False,6,4.0,30.42
10,VIST,41.34,False,4,3.0,40.13


Realizo los calculos necesarios para obtener mis rendimientos

In [34]:
# Calculo con los cedears
df_cedears = df_cedears.drop(['es_pesos'], axis=1)
df_cedears['Total_invertido'] = ((df_cedears['PPC'] / df_cedears['ratio']) * df_cedears['Cantidad']).round(2)
df_cedears['Tenencia_actual'] = ((df_cedears['Precio_Cierre'] / df_cedears['ratio']) * df_cedears['Cantidad']).round(2)
df_cedears['Neto'] = (df_cedears['Tenencia_actual'] - df_cedears['Total_invertido'])
df_cedears['Rendimiento'] = (((df_cedears['Tenencia_actual'] - df_cedears['Total_invertido']) / df_cedears['Total_invertido']) * 100).round(2)
df_cedears.sort_values('Rendimiento', ascending=False, inplace=True)

# Calculo mi tenencia actual total y rendimiento toal en cedears
total_cedears = df_cedears['Tenencia_actual'].sum()
rend_tot_ced = (((df_cedears['Tenencia_actual'].sum() - df_cedears['Total_invertido'].sum()) / df_cedears['Total_invertido'].sum()) * 100).round(2)

print(f'La tenencia actual en CEDEARS es de ${total_cedears}, y el rendimiento total es del {rend_tot_ced}%')

La tenencia actual en CEDEARS es de $584.27, y el rendimiento total es del 7.13%


Realizo lo mismo pero para sacar el rendimiento de las acciones

In [ ]:
dolar_ej = 1296
df_acciones['Total_invertido'] = (df_acciones['PPC'] * df_acciones['Cantidad']).round(2)

Se realiza una descarga en excel de los datos.

In [ ]:
# Guardo un excel con los datos
df_tickers.to_excel(f'informe-acciones-usd.xlsx')

# Visualización
Utilizando la libreria de plotly se realizan 2 gráficos para saber la participación de cada acción en nuestra cartera y para comparar lo total invertido de cada acción vs lo total ganado en cada acción.

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

# Calculo la ganancia total en USD
total_usd = (df_tickers['Gan/Perd USD'].sum()).round(2)

# Grafico la participación x accion en usd
fig_pie = px.pie(df_tickers, values='Total invertido USD', names='Ticker', title='Participación en USD por acción')
fig_pie.update_traces(textposition='outside', textinfo='percent+label')
fig_pie.add_annotation(text=f'La ganancia neta es de {total_usd} USD', x=-0.3, y=1.1, showarrow=False, font=dict(size=10, color='gray'), xref='paper', yref='paper')
fig_pie.show()

# Ganancia/perdida en dolares de cada accion
fig_bars = go.Figure(data=[
    go.Bar(name='Total invertido USD', x=df_tickers['Ticker'], y=df_tickers['Total invertido USD']),
    go.Bar(name='Gan/Perd USD', x=df_tickers['Ticker'], y=df_tickers['Gan/Perd USD'])
])
# Change the bar mode
fig_bars.update_layout(barmode='group', title='Total invertido y Ganancias en USD por accion')
fig_bars.show()